# Trainers

Designing a training loop for learning a complex dataset.

In [ ]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils as utils
from torchvision import datasets, transforms

from lib.dataset import PCam
from lib.evaluate import evaluate
from lib.models import Veggie16
from lib.train import train

In [ ]:
DATASET_PATH = '../data'

In [ ]:
image_dir = os.path.join(DATASET_PATH, 'train')
csv_path = os.path.join(DATASET_PATH, 'train_labels.csv')
pcam_dataset = PCam(image_dir, csv_path, 
                    transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.5,),(0.5,))
                    ]))
print(len(pcam_dataset))

In [ ]:
train_set, val_set = utils.data.random_split(pcam_dataset, [154000, 66025])

In [ ]:
train_loader = utils.data.DataLoader(train_set, batch_size=4, num_workers=0)
val_loader = utils.data.DataLoader(val_set, batch_size=4, num_workers=0)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

In [ ]:
num_epochs = 2
num_classes = 2
batch_size = 100
learning_rate = 0.001

In [ ]:
model = Veggie16(2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
print(model)

In [ ]:
losses = train(model, train_loader, device, criterion, optimizer, num_epochs=num_epochs)

In [ ]:
plt.semilogy()
plt.plot(tuple(range(num_epochs)), losses, 'b-')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylable('Cross Entropy Loss')
plt.show()

In [ ]:
score, accuracy, loss = evaluate(model, val_loader, device, criterion)
print('Evaluating Veggie16 on Validation set:')
print('F1-Score:', score)
print('Accuracy:', accuracy)
print('Loss:', loss)